In [2]:
# Suppression des warnings
import warnings
warnings.filterwarnings('ignore')
# Import des librairies
import pandas as pd
import numpy as np

In [14]:
# Ouverture du csv (entreprises,geo,population,salaires)
all_content_2014 = pd.read_csv ('all_content_2014.csv',index_col=0)

In [51]:
# Code région converti en string
all_content_2014.REG = all_content_2014.REG.astype(str)
all_content_2014.code_région = all_content_2014.code_région.astype(str)
# Affichage des infos
all_content_2014.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 36713 entries, 0 to 36712
Data columns (total 51 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   CODGEO              36713 non-null  object 
 1   LIBGEO              36713 non-null  object 
 2   REG                 36713 non-null  object 
 3   DEP                 36713 non-null  object 
 4   EU_circo            36713 non-null  object 
 5   code_région         36713 non-null  object 
 6   nom_région          36713 non-null  object 
 7   chef.lieu_région    36713 non-null  object 
 8   numéro_département  36713 non-null  object 
 9   nom_département     36713 non-null  object 
 10  préfecture          36713 non-null  object 
 11  nom_commune         36713 non-null  object 
 12  code_insee          36713 non-null  object 
 13  latitude            33770 non-null  float64
 14  longitude           33855 non-null  float64
 15  éloignement         33767 non-null  float64
 16  NB  

In [61]:
# Ouverture du csv (Vivien données geo manquantes)
df7 = pd.read_csv('geo_introuvables.csv')

In [82]:
# Conversion du type CODGEO, REG et DEP en str
df7.CODGEO = df7.CODGEO.astype(str)
df7.REG = df7.REG.astype(str)
df7.DEP = df7.DEP.astype(str)
# Affichage des infos
df7.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21 entries, 0 to 20
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   CODGEO            21 non-null     object 
 1   LIBGEO            21 non-null     object 
 2   REG               21 non-null     object 
 3   DEP               21 non-null     object 
 4   Total_ets         21 non-null     int64  
 5   0/NA              21 non-null     int64  
 6   1 à 5             21 non-null     int64  
 7   6 à 9             21 non-null     int64  
 8   10 à 19           21 non-null     int64  
 9   20 à 49           21 non-null     int64  
 10  50 à 99           21 non-null     int64  
 11  100 à 199         21 non-null     int64  
 12  200 à 499         21 non-null     int64  
 13  500 et +          21 non-null     int64  
 14  EU_circo          21 non-null     object 
 15  nom_région        21 non-null     object 
 16  nom_département   21 non-null     object 
 17 

In [83]:
# Extraction des départements de la liste des dupliqués
departements = df7.DEP.unique()
# Affichage des départements
departements

array(['26', '31', '35', '52', '55', '61', '62', '88', '89', '976'],
      dtype=object)

In [66]:
# Création d'une dataframe
df_dep=pd.DataFrame(columns=['REG','DEP','EU_circo','code_région','nom_région','chef.lieu_région',
                             'nom_département','préfecture'])

# Recherche des départements dans la DF all_content
for dep in departements:
    # Extraction des lignes contenant le numero de département
    df_temp = all_content_2014.loc[all_content_2014.DEP==dep]
    # On selectionne la première ligne
    df_temp = df_temp.iloc[0]
    # Ajout à la DF créee
    df_dep = df_dep.append(df_temp[['REG','DEP','EU_circo','code_région','nom_région','chef.lieu_région',
                             'nom_département','préfecture']])

# Affichage des lignes contenant les infos des numéros de département concernés
df_dep.head()

,REG,DEP,EU_circo,code_région,nom_région,chef.lieu_région,nom_département,préfecture
9335,82,26,Sud-Est,82,Rhône-Alpes,Lyon,Drôme,Valence
11756,73,31,Sud-Ouest,73,Midi-Pyrénées,Toulouse,Haute-Garonne,Toulouse
13686,53,35,Ouest,53,Bretagne,Rennes,Ille-et-Vilaine,Rennes
36564,21,52,Est,21,Champagne-Ardenne,Châlons-en-Champagne,Haute-Marne,Chaumont
36577,41,55,Est,41,Lorraine,Metz,Meuse,Bar-le-Duc


In [84]:
# Création d'une DF temporaire
temp_new = pd.DataFrame()
# on parcourt la DF obtenue precedemment contenant les infos par département
for i in range(len(df_dep)):
    # extraction des lignes dupliquées contenant le numéro de département
    temp = df7.loc[df7.DEP==df_dep.iloc[i,1]]
    # Remplissage des lignes avec les infos manquantes du département
    temp[['REG','DEP','EU_circo','code_région','nom_région','chef.lieu_région','nom_département','préfecture']] = df_dep.iloc[i]
    # Ajout à la DF temporaire
    temp_new = temp_new.append(temp)
# Affichage de la DF contenant les infos à compléter
temp_new.head(3)

,CODGEO,LIBGEO,REG,DEP,Total_ets,0/NA,1 à 5,6 à 9,10 à 19,20 à 49,...,200 à 499,500 et +,EU_circo,nom_région,nom_département,latitude,longitude,chef.lieu_région,préfecture,code_région
0,26020,La Répara-Auriples,82,26,24,22,2,0,0,0,...,0,0,Sud-Est,Rhône-Alpes,Drôme,NaN,NaN,Lyon,Valence,82
1,31300,Lieoux,73,31,2,2,0,0,0,0,...,0,0,Sud-Ouest,Midi-Pyrénées,Haute-Garonne,NaN,NaN,Toulouse,Toulouse,73
2,35317,Saint-Symphorien,53,35,15,6,4,2,2,1,...,0,0,Ouest,Bretagne,Ille-et-Vilaine,NaN,NaN,Rennes,Rennes,53


In [79]:
# On complete la DF df7 avec les infos precedemment extraites
df7[df7.columns[14:17]] = df7[df7.columns[14:17]].fillna(value=temp_new[temp_new.columns[14:17]])
df7['chef.lieu_région'] = temp_new['chef.lieu_région']
df7['préfecture'] = temp_new['préfecture']
# Affichage de la DF
df7

,CODGEO,LIBGEO,REG,DEP,Total_ets,0/NA,1 à 5,6 à 9,10 à 19,20 à 49,...,100 à 199,200 à 499,500 et +,EU_circo,nom_région,nom_département,latitude,longitude,chef.lieu_région,préfecture
0,26020,La Répara-Auriples,82,26,24,22,2,0,0,0,...,0,0,0,Sud-Est,Rhône-Alpes,Drôme,NaN,NaN,Lyon,Valence
1,31300,Lieoux,73,31,2,2,0,0,0,0,...,0,0,0,Sud-Ouest,Midi-Pyrénées,Haute-Garonne,NaN,NaN,Toulouse,Toulouse
2,35317,Saint-Symphorien,53,35,15,6,4,2,2,1,...,0,0,0,Ouest,Bretagne,Ille-et-Vilaine,NaN,NaN,Rennes,Rennes
3,52033,Avrecourt,21,52,4,3,0,0,1,0,...,0,0,0,Est,Champagne-Ardenne,Haute-Marne,NaN,NaN,Châlons-en-Champagne,Chaumont
4,52124,Chézeaux,21,52,2,1,1,0,0,0,...,0,0,0,Est,Champagne-Ardenne,Haute-Marne,NaN,NaN,Châlons-en-Champagne,Chaumont
5,52266,Laneuville-à-Rémy,21,52,4,2,2,0,0,0,...,0,0,0,Est,Champagne-Ardenne,Haute-Marne,NaN,NaN,Châlons-en-Champagne,Chaumont
6,52278,Lavilleneuve-au-Roi,21,52,4,3,1,0,0,0,...,0,0,0,Est,Champagne-Ardenne,Haute-Marne,NaN,NaN,Châlons-en-Champagne,Chaumont
7,52465,Saulxures,21,52,2,1,0,1,0,0,...,0,0,0,Est,Champagne-Ardenne,Haute-Marne,NaN,NaN,Châlons-en-Champagne,Chaumont
8,55039,Beaumont-en-Verdunois,41,55,0,0,0,0,0,0,...,0,0,0,Est,Lorraine,Meuse,NaN,NaN,Metz,Bar-le-Duc
9,55050,Bezonvaux,41,55,0,0,0,0,0,0,...,0,0,0,Est,Lorraine,Meuse,NaN,NaN,Metz,Bar-le-Duc


In [81]:
# Affichage des infos
df7.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21 entries, 0 to 20
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   CODGEO            21 non-null     object 
 1   LIBGEO            21 non-null     object 
 2   REG               21 non-null     object 
 3   DEP               21 non-null     object 
 4   Total_ets         21 non-null     int64  
 5   0/NA              21 non-null     int64  
 6   1 à 5             21 non-null     int64  
 7   6 à 9             21 non-null     int64  
 8   10 à 19           21 non-null     int64  
 9   20 à 49           21 non-null     int64  
 10  50 à 99           21 non-null     int64  
 11  100 à 199         21 non-null     int64  
 12  200 à 499         21 non-null     int64  
 13  500 et +          21 non-null     int64  
 14  EU_circo          21 non-null     object 
 15  nom_région        21 non-null     object 
 16  nom_département   21 non-null     object 
 17 

In [85]:
# Sauvegarde de la DF all_content_pop dans un fichier csv avec encodage windows
df7.to_csv("geo_introuvables_completes.csv",encoding= "utf-8")